In [ ]:
import numpy as np
import gennoisy
import matplotlib.pylab as plt
import pandas as pd

%matplotlib inline

In [ ]:
def sigmoid(X):
    return 1./(1 + np.exp(-X))

def dsigmoid(X):
    return sigmoid(X)*(1 - sigmoid(X))

In [ ]:
X,S,A,N = gennoisy.gennoisy()
plt.plot(X); plt.title('Simulated microphone receiver')

### Pre-whiten the data


The sample covariance matrix is calculated:
$$ S_c = \frac{1}{N} \left( X - \mu_x \right)^T (X - \mu_x ) $$

Whitened data is then:
$$ \hat{X} = \Phi \Lambda^{-0.5} \Phi^T (X - \mu_x) $$

where 

$$ S_c = \Phi \Lambda \Phi^T $$

In [ ]:
Xm = (X - X.mean(axis=0))
N = X.shape[0]

Sc = Xm.T @ Xm / N
[eigvals, eigvecs] = np.linalg.eig(Sc)

eigwhite = eigvecs*(eigvals**-0.5).dot(eigvecs.T)
Xw = eigwhite.dot( Xm.T )

### Nonlinearities


$$ f(u) = - e^{-u^2/2} $$
$$ g(u) = u e^{-u^2/2} $$
$$ g'(u) = (1 - u^2) e^{-u^2/2} $$

In [ ]:
def f(u):
    return - np.exp(-u**2/2 )
def g(u):
    return u*np.exp( -u**2 / 2 )
def gp(u):
    return (1 - u**2)*np.exp(-u**2 / 2)

### Multiple component extraction

Per the wikipedia page at "https://en.wikipedia.org/wiki/FastICA"

In [ ]:
W =np.random.rand(3,3)
C = 3

for p in range(C):

    # Take the pth column of W
    wp = W[:,p]
    
    for i in range(1000):

        # Maximize non-Gaussianity
        wp = 1/N*Xw.dot(g(wp.dot(Xw)).T ) - gp(wp.dot(Xw)).sum()/N*wp 

        # Orthogonalize to other components
        for j in range(p):
            wj = W[:,j]
            wp = wp - wp.dot(wj)*wj

        # Reproject back to the unit norm
        wp = wp / np.linalg.norm(wp)

        # Reassign to W
        W[:,p] = wp
        
print(W)

In [ ]:
Sh = W.T.dot(Xm.T).T
plt.plot(Sh)